In [1]:
import json
import csv
import pandas as pd
import numpy as np
import scipy as sci
from scipy import sparse
from sklearn.model_selection import train_test_split
import sklearn.feature_extraction.text as sk_text
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score

In [2]:
#Open business.json file, create tsv file with business_id, business name, categories, and review count to be used as features 
#and stars as label

outfile = open("business.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','categories', 'stars', 'review_count'])
with open('yelp_academic_dataset_business.json', encoding="utf8") as f:
    for line in f:
        row = json.loads(line)
        sfile.writerow([row['business_id'], row['categories'], row['stars'],row['review_count'] ])

outfile.close()

business_df= pd.read_csv('business.tsv', delimiter ="\t")

In [3]:
#Open review.json file, create tsv file with business_id,text to be used as features 
#and stars as label

outfile = open("review_stars.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','stars', 'text'])
with open('yelp_academic_dataset_review.json', encoding="utf8") as f:
    for line in f:
        row = json.loads(line)
        # some special char must be encoded in 'utf-8'
        sfile.writerow([row['business_id'], row['stars'], (row['text']).encode('utf-8')])

outfile.close()

review_df= pd.read_csv('review_stars.tsv', delimiter ="\t")

In [4]:
# Group all reviews by business_id
review_agg_df = review_df.groupby('business_id')['text'].sum()
df_ready_for_sklearn = pd.DataFrame({'business_id': review_agg_df.index, 'all_reviews': review_agg_df.values})


In [5]:
#Merge the resulting review aggregate dataframe with business dataframe
merge_df = pd.merge(business_df, df_ready_for_sklearn, on='business_id')


In [6]:
#print the first 5 rows using head
merge_df.head()

,business_id,categories,stars,review_count,all_reviews
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",4.0,24,b'Great place. Major flaw is how early it clos...
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",4.5,3,"b""CK's BBQ is off the charts best BBQ I have E..."
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",4.0,5,"b""La nourriture est excellente, le service Imp..."
3,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",1.5,8,b'GEICO for auto is great! But they are really...
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",2.0,4,"b""This listing was originally under gardening ..."


In [7]:
#Normalization of review count field so it becomes comparable and remove bias
merge_df.insert(3,'normalized_count',((merge_df['review_count'] - merge_df['review_count'].min()) / (merge_df['review_count'].max() - merge_df['review_count'].min())).astype(float))
merge_df.drop('review_count', axis=1, inplace=True)

In [8]:
#print the normalized counts for first 5 rows
merge_df.head()

,business_id,categories,stars,normalized_count,all_reviews
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",4.0,0.002637,b'Great place. Major flaw is how early it clos...
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",4.5,0.000000,"b""CK's BBQ is off the charts best BBQ I have E..."
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",4.0,0.000251,"b""La nourriture est excellente, le service Imp..."
3,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",1.5,0.000628,b'GEICO for auto is great! But they are really...
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",2.0,0.000126,"b""This listing was originally under gardening ..."


In [9]:
#TF-IDF calculation

tfidf = sk_text.TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))

In [10]:
# Fit the reviews column with TFIDFvectorizer
matrix = tfidf.fit_transform(merge_df['all_reviews'])
matrix = matrix.toarray()

In [11]:
# We are adding the normalized count to the original matrix with TFIDFvectorizer
x_matrix=np.column_stack((matrix, merge_df['normalized_count']))

In [12]:
#train test data for linear regression

x_train, x_test, y_train, y_test = train_test_split(x_matrix, merge_df['stars'] , test_size=0.2, random_state=42)

In [13]:
# Checking the shape of train and test data
x_train.shape

(150874, 1001)

In [14]:
x_test.shape


(37719, 1001)

In [15]:
y_train.shape

(150874,)

In [16]:
y_test.shape

(37719,)

In [17]:
# linear regression

lin_reg_model = LinearRegression()

lin_reg_model.fit(x_train, y_train)

y_pred = lin_reg_model.predict(x_test)

y_pred

array([2.79065971, 4.04537282, 4.74927598, ..., 2.42110101, 3.49581221,
       2.88075762])

In [ ]:
# list  the business with the stars and prediction

for i in range(0,10):
    idx=y_test.index[i]
    print("business id - %s actual stars label - %d predicted - %d" 
          %(merge_df['business_id'][idx], y_test[idx], y_pred[i]))

In [18]:
#Performance of Linear Regression Model
# The lower the mean squared the better performance
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
print('R2 score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 0.27
R2 score: 0.74


In [19]:
#stars need to be encoded for all the models we are going to train
le = preprocessing.LabelEncoder()
merge_df['encoded_stars'] = le.fit_transform(merge_df['stars'])

In [20]:
#Display encoded stars
merge_df.head()

,business_id,categories,stars,normalized_count,all_reviews,encoded_stars
0,Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",4.0,0.002637,b'Great place. Major flaw is how early it clos...,6
1,AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",4.5,0.000000,"b""CK's BBQ is off the charts best BBQ I have E...",7
2,O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",4.0,0.000251,"b""La nourriture est excellente, le service Imp...",6
3,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",1.5,0.000628,b'GEICO for auto is great! But they are really...,1
4,8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",2.0,0.000126,"b""This listing was originally under gardening ...",2


In [21]:
# train test dat afor other models

x_train1, x_test1, y_train1, y_test1 = train_test_split(x_matrix, merge_df['encoded_stars'] , test_size=0.2, random_state=42)

In [22]:
# logistic Regression

Log_reg_model = LogisticRegression()

Log_reg_model.fit(x_train1, y_train1)

y_pred1 = Log_reg_model.predict(x_test1)

y_pred1

array([4, 6, 8, ..., 4, 5, 5], dtype=int64)

In [23]:
# list  the business with the stars and prediction

for i in range(0,10):
    idx=y_test1.index[i]
    print("business id - %s actual stars label - %d predicted - %d" 
          %(merge_df['business_id'][idx], y_test1[idx], y_pred1[i]))

business id - y6cfbFoQuPBf4H6mubticw actual stars label - 3 predicted - 4
business id - tVW7pefO8xIgWMF86oVhKQ actual stars label - 6 predicted - 6
business id - WYSaOdpqFAV9NxrVdKKR9w actual stars label - 8 predicted - 8
business id - MJjTgjRZJLnU0VHF1TeMGQ actual stars label - 5 predicted - 5
business id - 5bwIr6lySrlr2fUQJEoqkg actual stars label - 8 predicted - 8
business id - ZYyRkdrSKG7tYwuE8R38og actual stars label - 6 predicted - 5
business id - T1iskW4W6jGRTwzq3DKN_Q actual stars label - 7 predicted - 8
business id - 83lCWOhpSJ0DcTczGfGcxg actual stars label - 6 predicted - 8
business id - SpO3kyBfxN0wpfdkLSNepA actual stars label - 8 predicted - 8
business id - Cbu0DpFdk47f7FUwNku7-A actual stars label - 8 predicted - 8


In [24]:
#Performance of Linear Regression Model
# The lower the mean squared the better performance
print("Mean squared error: %.2f"
      % mean_squared_error(y_test1, y_pred1))
print('R2 score: %.2f' % r2_score(y_test1, y_pred1))

Mean squared error: 1.46
R2 score: 0.64


In [ ]:
#implementing Nearest Neighbor

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(x_train1, y_train1) 

y_pred1 = knn.predict(x_test1)

y_pred1 

In [ ]:
# list  the business with the stars and prediction

for i in range(0,10):
    idx=y_test1.index[i]
    print("business id - %s actual stars label - %d predicted - %d" 
          %(merge_df['business_id'][idx], y_test1[idx], y_pred1[i]))

In [ ]:
print("accuracy: %.2f"
      % accuracy_score(y_test1, y_pred1))

In [ ]:
# SVM

svm_model = SVC()

svm_model.fit(x_train1, y_train1)

y_pred1 = svm_model.predict(x_test1)

y_pred1

In [ ]:
# list  the business with the stars and prediction

for i in range(0,10):
    idx=y_test1.index[i]
    print("business id - %s actual stars label - %d predicted - %d" 
          %(merge_df['business_id'][idx], y_test1[idx], y_pred1[i]))

In [ ]:
print("accuracy: %.2f"
      % accuracy_score(y_test1, y_pred1))

In [ ]:
# MNB

mnb_model = MultinomialNB()

mnb_model.fit(x_train1, y_train1)

y_pred1 = mnb_model.predict(x_test1)

y_pred1

In [ ]:
# list  the business with the stars and prediction

for i in range(0,10):
    idx=y_test1.index[i]
    print("business id - %s actual stars label - %d predicted - %d" 
          %(merge_df['business_id'][idx], y_test1[idx], y_pred1[i]))

In [ ]:
print("accuracy: %.2f"
      % accuracy_score(y_test1, y_pred1))

** Additional Features **

** POSTAL CODE **

In [ ]:
# creating new TSV for business dataset adding postal code as an additional feature

outfile = open("business_postal.tsv", 'w')
sfile = csv.writer(outfile, delimiter ="\t", quoting=csv.QUOTE_MINIMAL)
sfile.writerow(['business_id','categories', 'stars', 'review_count','postal code'])
with open('yelp_academic_dataset_business.json') as f:
    for line in f:
        row = json.loads(line)
        # some special char must be encoded in 'utf-8'
        sfile.writerow([row['business_id'], row['categories'], row['stars'],row['review_count'], row['postal_code']])

outfile.close()

In [ ]:
# Creating new dataframe

business_postal_df= pd.read_csv('business_nei.tsv', delimiter ="\t")

In [ ]:
# merging the new dataframe with the reviews
merge_postal_df = pd.merge(business_postal_df, df_ready_for_sklearn, on='business_id')

In [ ]:
# one-hot cooding of postal codes 

hotcoded_df = pd.get_dummies(merge_postal_df['postal code'], sparse = 'true')

In [ ]:
# printing the One Hot Coded postal codes

hotcoded_df.head()

In [ ]:
# merging the coded postal codes with the main dataframe

result_df = pd.concat([merge_postal_df, hotcoded_df], axis=1, sort=False)

In [ ]:
# merge the encoded postal codes with the matrix of TFIDF and Review count

x_matrix=np.column_stack((x_matrix,hotcoded_df))

In [ ]:
# Spliting the new data set

x_train2, x_test2, y_train2, y_test2 = train_test_split(x_matrix, merge_df['encoded_stars'] , test_size=0.2, random_state=42)

In [ ]:
# MNB

mnb_model = MultinomialNB()

mnb_model.fit(x_train2, y_train2)

y_pred2 = mnb_model.predict(x_test2)

y_pred2

In [ ]:
print("accuracy: %.2f"
      % accuracy_score(y_test2, y_pred2))

In [ ]:
# SVM

svm_model = SVC(kernel="linear")

svm_model.fit(x_train2, y_train2)

y_pred2 = svm_model.predict(x_test2)

y_pred2

In [ ]:
print("accuracy: %.2f"
      % accuracy_score(y_test2, y_pred2))

** CATEGORIES **

In [ ]:
# Preparing category field for one hot coding

merge_df['categories'].str.split(',')

In [ ]:
# Extraction the word from a list for a given position

def get_element(my_list, position):
   return my_list[position]

In [ ]:
# calling the get_element function for each row of dataframe for extecting categories  

merge_df['categories'] = merge_df['categories'].apply(get_element, position=0)

In [ ]:
# one hot coding of categories

hotcoded_df = pd.get_dummies(merge_df['categories'], sparse = 'true')

In [ ]:
# merge the one hot coded categories with the matrix of TFIDF and Review count

x_matrix = np.column_stack((x_matrix, hotcoded_df))

In [ ]:
# Spliting the new data set

x_train3, x_test3, y_train3, y_test3 = train_test_split(x_matrix, merge_df['encoded_stars'] , test_size=0.2, random_state=42)

In [ ]:
# MNB

mnb_model = MultinomialNB()

mnb_model.fit(x_train3, y_train3)

y_pred3 = mnb_model.predict(x_test3)

y_pred3

In [ ]:
print("accuracy: %.2f"
      % accuracy_score(y_test3, y_pred3))

In [ ]:
# SVM

svm_model = SVC(kernel="linear")

svm_model.fit(x_train3, y_train3)

y_pred3 = svm_model.predict(x_test3)

y_pred3

In [ ]:
print("accuracy: %.2f"
      % accuracy_score(y_test3, y_pred3))